In [ ]:


# 1. Classifier: LLM that classifies a natural language query into one of three categories: PnL, Balance sheets, stocks
# Three different paths: Each a separate node in the graph

# 2.
# PnL: Retrieve from a PnL vector DB
# Balance sheets: Retrieve from a balance sheet vector DB
# Stocks: Retrieve from a stock vector DB

# Attach retrieved documents to the next step:

# 3.Natural Language to SQL: LLM that converts the natural language query + retrieved documents into a SQL query

# Output is the SQL query


In [4]:
from typing import Annotated

from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, HumanMessage
from typing_extensions import TypedDict
from langchain_core.utils.function_calling import convert_to_openai_tool
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from dotenv import load_dotenv
from judgeval.common.tracer import Tracer, wrap, JudgevalCallbackHandler
from judgeval.scorers import FaithfulnessScorer, AnswerRelevancyScorer, AnswerCorrectnessScorer

import asyncio
import os
from typing import Any, Optional
from uuid import UUID

import openai
import os
import asyncio
from tavily import TavilyClient
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions

ModuleNotFoundError: No module named 'judgeval'

In [ ]:
def populate_vector_db(collection, destinations_data):
    """
    Populate the vector DB with travel information.
    destinations_data should be a list of dictionaries with 'destination' and 'information' keys
    """
    for data in destinations_data:
        collection.add(
            documents=[data['information']],
            metadatas=[{"destination": data['destination']}],
            ids=[f"destination_{data['destination'].lower().replace(' ', '_')}"]
        )

def initialize_vector_db():
    """Initialize ChromaDB with OpenAI embeddings."""
    client = chromadb.Client()
    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        model_name="text-embedding-3-small"
    )
    res = client.get_or_create_collection(
        "travel_information",
        embedding_function=embedding_fn
    )
    populate_vector_db(res, destinations_data)
    return res

In [3]:
judgment = Tracer(api_key=os.getenv("JUDGMENT_API_KEY"))

# ... existing code ...

async def main():
    with judgment.trace(
        "langgraph_run1",
        project_name="langgraph_basic",
        overwrite=True
    ) as trace:
        # Define our state type
        class AgentState(TypedDict):
            messages: list[BaseMessage]
            category: Optional[str]

        # Initialize the graph
        graph_builder = StateGraph(AgentState)
        
        # Create the classifier node with a system prompt
        classifier = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
            system_message="""You are a financial query classifier. Your job is to classify user queries into one of three categories:
            - 'pnl' for Profit and Loss related queries
            - 'balance_sheets' for Balance Sheet related queries
            - 'stocks' for Stock market related queries
            
            Respond ONLY with the category name in lowercase, nothing else."""
        )
        
        # Add classifier node
        graph_builder.add_node("classifier", classifier)
        
        # Add router node to direct flow based on classification
        def router(state: AgentState) -> str:
            return state["category"]
        
        # Add conditional edges based on classification
        graph_builder.add_conditional_edges(
            "classifier",
            router,
            {
                "pnl": "pnl_retriever",
                "balance_sheets": "balance_sheet_retriever",
                "stocks": "stock_retriever"
            }
        )
        
        # Add retriever nodes (placeholder functions for now)
        graph_builder.add_node("pnl_retriever", lambda x: {"messages": x["messages"], "documents": "PnL docs"})
        graph_builder.add_node("balance_sheet_retriever", lambda x: {"messages": x["messages"], "documents": "Balance sheet docs"})
        graph_builder.add_node("stock_retriever", lambda x: {"messages": x["messages"], "documents": "Stock docs"})

        # Compile the graph
        graph = graph_builder.compile()
        
    response = await graph.ainvoke({
        "messages": [HumanMessage(content="What was Apple's profit last quarter?")],
        "category": None
    })
    
    print(f"Response: {response=}")
    
        
        

NameError: name 'Tracer' is not defined